In [2]:
import { load } from "dotenv";
const env = await load();

const process = { env };
process.env;


{
  OPENAI_API_KEY: "sk-b72b74abd3f541a08b68dc756de84958",
  MODEL_NAME: "qwen-turbo-1101",
  BASE_URL: "https://dashscope.aliyuncs.com/compatible-mode/v1",
  DEEPSEEK_API_KEY: "sk-b1a4cf12df35436f972ba72d4e7c669b",
  EMBEDDING_MODEL_NAME: "text-embedding-v1",
  AZURE_OPENAI_API_KEY: "923274d2ec144d95b33adad2c29a362e",
  AZURE_OPENAI_API_INSTANCE_NAME: "gemdalechatgpt4",
  AZURE_OPENAI_API_VERSION: "2023-07-01-preview",
  AZURE_OPENAI_API_DEPLOYMENT_NAME: "jdkjkjgpt4turbo",
  AZURE_EMBEDDING_MODEL_NAME: "text-embedding-ada-002",
  SERP_API_KEY: "185032b32dc536d633129d221f7f7be48629f0e4569138241d817449166e21ca"
}

In [3]:
import OpenAI from "openai";

const openai = new OpenAI({
  apiKey: process.env.OPENAI_API_KEY,
  baseURL: process.env.BASE_URL,
});

function getCurrentWeather({ location, unit = "fahrenheit" }) {
  const weatherInfo = {
    location,
    temperature: 72,
    unit,
    forecast: ["sunny", "windy"],
  };
  return JSON.stringify(weatherInfo);
}

const tools = [
  {
    type: "function",
    function: {
      name: "getCurrentWeather",
      description: "Get the current weather in a given location",
      parameters: {
        type: "object",
        properties: {
          location: {
            type: "string",
            description: "The city and state, e.g. San Francisco, CA",
          },
          unit: {
            type: "string",
            enum: ["celsius", "fahrenheit"],
            description: "The unit of temperature",
          },
        },
        required: ["location"],
      },
    },
  },
];
const messages = [
  { role: "user", content: "北京的天气怎么样？" },
  //   { role: "user", content: "What's the weather like in Beijing?" },
];

const res = await openai.chat.completions.create({
  model: process.env.MODEL_NAME,
  messages,
  tools,
  //   tool_choice: "none"
  tool_choice: "auto",
  //   tool_choice: {
  //     type: "function",
  //     function: {
  //       name: "getCurrentWeather",
  //     },
  //   },
});

// console.log(res.choices[0]);

const cell = res.choices[0].message.tool_calls[0];
const functionInfo = cell.function;
const functionName = functionInfo.name;
const functionParams = functionInfo.arguments;

const functions = { getCurrentWeather };

const functionResult = functions[functionName](functionParams);

messages.push(res.choices[0].message);
messages.push({
  tool_call_id: cell.id,
  role: "tool",
  name: functionName,
  content: functionResult,
});

// console.log(messages);

await openai.chat.completions.create({
  model: process.env.MODEL_NAME,
  messages,
});


{
  choices: [
    {
      message: {
        content: "当前北京的天气情况是晴朗且多风，温度为72华氏度，换算成摄氏度大约是22摄氏度。",
        role: "assistant"
      },
      finish_reason: "stop",
      index: 0,
      logprobs: null
    }
  ],
  object: "chat.completion",
  usage: { prompt_tokens: 43, completion_tokens: 34, total_tokens: 77 },
  created: 1746695737,
  system_fingerprint: null,
  model: "qwen-turbo-1101",
  id: "chatcmpl-e7442bbf-ba05-94a5-9a0e-f24c75c75020"
}

In [4]:
import { z } from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";

const getCurrentWeatherSchema = z.object({
  location: z.string().describe("The city and state, e.g. San Francisco, CA"),
  unit: z.enum(["celsius", "fahrenheit"]).describe("The unit of temperature"),
});

const getCurrentTimeSchema = z.object({
  format: z
    .enum(["iso", "locale", "string"])
    .optional()
    .describe("The format of the time, e.g. iso, locale, string"),
});

// console.log(zodToJsonSchema(getCurrentTimeSchema))

const model = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  temperature: 0,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const modelWithTools = model.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: zodToJsonSchema(getCurrentWeatherSchema),
      },
    },
    {
      type: "function",
      function: {
        name: "getCurrentTime",
        description: "Get the current time",
        parameters: zodToJsonSchema(getCurrentTimeSchema),
      },
    },
  ],
  tool_choice: "auto",
});

// await modelWithTools.invoke('北京的天气怎么样')

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant"],
  ["human", "{input}"],
]);

const chain = prompt.pipe(modelWithTools).pipe(new JsonOutputToolsParser());

await chain.invoke({
  input: "北京天气怎么样",
});


[
  {
    type: "getCurrentWeather",
    args: { location: "北京", unit: "celsius" },
    id: undefined
  }
]

In [5]:
import { z } from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";

const taggingSchema = z.object({
  emotion: z.enum(["pos", "neg", "neutral"]).describe("文本的情感"),
  language: z.string().describe("文本的核心语言（应为ISO 639-1代码）"),
});

const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: `https://${process.env.AZURE_OPENAI_API_INSTANCE_NAME}.openai.azure.com/openai/deployments/${process.env.AZURE_OPENAI_API_DEPLOYMENT_NAME}`,
    apiKey: process.env.AZURE_OPENAI_API_KEY,
    defaultQuery: {
      "api-version": process.env.AZURE_OPENAI_API_VERSION,
    },
  },
});

const modelWithTools = chatModel.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "tagging",
        description: "对文本进行情感分析和语言检测",
        parameters: zodToJsonSchema(taggingSchema),
      },
    },
  ],
  tool_choice: {
    type: "function",
    function: {
      name: "tagging",
    },
  },
});

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "仔细思考，你有充足的时间进行严谨的思考，然后按照指示对文本进行标记",
  ],
  ["human", "{input}"],
]);

const chain = prompt.pipe(modelWithTools).pipe(new JsonOutputToolsParser());

await chain.invoke({
  input: "メリークリスマス!",
});


[
  {
    type: "tagging",
    args: { emotion: "pos", language: "ja" },
    id: undefined
  }
]

In [6]:
import { z } from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";

const personExtractionSchema = z
  .object({
    name: z.string().describe("人的名字"),
    age: z.number().optional().describe("人的年龄"),
  })
  .describe("提取关于一个人的信息");

const relationExtractSchema = z.object({
  people: z.array(personExtractionSchema).describe("提取所有人"),
  relation: z.string().describe("人之间的关系, 尽量简洁"),
});

zodToJsonSchema(relationExtractSchema);

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const modelExact = chatModel.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "relationExtract",
        description: "提取数据中人的信息和人的关系",
        parameters: zodToJsonSchema(relationExtractSchema),
      },
    },
  ],
  tool_choice: {
    type: "function",
    function: {
      name: "relationExtract",
    },
  },
});

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "仔细思考，你有充足的时间进行严谨的思考，然后提取文中的相关信息，如果没有明确提供，请不要猜测，可以仅提取部分信息",
  ],
  ["human", "{input}"],
]);

const chain = prompt.pipe(modelExact).pipe(new JsonOutputToolsParser());

const res = await chain.invoke({
  input: "张三和李四是好朋友，他们的年龄分别是20和21",
});

console.log(res);


[
  {
    type: "relationExtract",
    args: {
      people: [ { name: "张三", age: 20 }, { name: "李四", age: 21 } ],
      relation: "好朋友"
    },
    id: undefined
  }
]


In [10]:
import { z } from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate, PromptTemplate } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";
import { RunnableSequence, RunnableBranch } from "@langchain/core/runnables";

const classifySchema = z.object({
  type: z.enum(["科普", "编程", "一般问题"]).describe("用户提问的分类"),
});

const model = new ChatOpenAI({
  temperature: 0,
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const modelWithTools = model.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "classifyQuestion",
        description: "对用户的提问进行分类",
        parameters: zodToJsonSchema(classifySchema),
      },
    },
  ],
  tool_choice: { type: "function", function: { name: "classifyQuestion" } },
});

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `仔细思考，你有充足的时间进行严谨的思考，然后对用户的问题进行分类，当你无法分类到特定分类时，可以分类到 "一般问题"`,
  ],
  ["human", "{input}"],
]);

const classifyChain = RunnableSequence.from([
  prompt,
  modelWithTools,
  new JsonOutputToolsParser(),
  (input) => {
    console.log(input);
    const type = input[0]?.args?.type;
    return type ? type : "一般问题";
  },
]);

// await classifyChain.invoke({
//   input: "用js实现数组排序",
// });

const answeringModel = new ChatOpenAI({
  temperature: 0.7,
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const sciencePrompt = PromptTemplate.fromTemplate(
  `作为一位科普专家，你需要解答以下问题，尽可能提供详细、准确和易于理解的答案：
  问题：{input}
  答案：`
);

const programmingPrompt = PromptTemplate.fromTemplate(
  `作为一位编程专家，你需要解答以下编程相关的问题，尽可能提供详细、准确和实用的答案：
  问题：{input}
  答案：`
);

const generalPrompt = PromptTemplate.fromTemplate(
  `作请回答以下一般性问题，尽可能提供全面和有深度的答案：
  问题：{input}
  答案：`
);

const scienceChain = RunnableSequence.from([
  sciencePrompt,
  answeringModel,
  new StringOutputParser(),
  {
    output: (input) => input,
    role: () => "科普专家",
  },
]);

const programmingChain = RunnableSequence.from([
  programmingPrompt,
  answeringModel,
  new StringOutputParser(),
  {
    output: (input) => input,
    role: () => "编程大师",
  },
]);

const generalChain = RunnableSequence.from([
  generalPrompt,
  answeringModel,
  new StringOutputParser(),
  {
    output: (input) => input,
    role: () => "通识专家",
  },
]);

const branch = RunnableBranch.from([
  [(input) => input.type.includes("科普"), scienceChain],
  [(input) => input.type.includes("编程"), programmingChain],
  generalChain,
]);

const outputTemplate = PromptTemplate.fromTemplate(
  `感谢您的提问，这是来自 {role} 的专业回答：
    
    {output}
    `
);

const finalChain = RunnableSequence.from([
  {
    type: classifyChain,
    input: (input) => input.input,
  },
  branch,
  (input) => outputTemplate.format(input),
]);


await finalChain.invoke({
    "input": "鲸鱼是哺乳动物么？"
})

[ { type: "classifyQuestion", args: { type: "科普" }, id: undefined } ]


"感谢您的提问，这是来自 科普专家 的专业回答：\n" +
  "    \n" +
  "    是的，鲸鱼是哺乳动物。这个问题的答案可以从几个方面来解释：\n" +
  "\n" +
  "1. **生物学分类**：在生物学上，鲸鱼属于哺乳纲（Mammalia）。哺乳纲包括了所有哺乳动物，如人类、狗、猫等。这一分类基于一系列共享特征，这些特征将哺乳动物与其他生物区别开来。\n" +
  "\n" +
  "2. **生理结构**：鲸鱼具有哺乳动物的典型生理特征，例如：\n" +
  "   - **乳腺**：母鲸鱼能够通过乳腺分泌乳汁喂养幼崽。\n" +
  "   - **毛发**：虽然成年鲸鱼体毛很少，但幼年鲸鱼身上有细小的毛发，这是哺乳动物的一个特征。\n" +
  "   - **温血性**：鲸鱼能够调节自己的体温，保持恒定的体温，这也是哺乳动物的一个特点。\n" +
  "   - **肺呼吸**：鲸鱼用肺呼吸空气，而不是像鱼类那样通过鳃呼吸水中的氧气。\n" +
  "\n" +
  "3. **行为习性**：鲸鱼的行为也符合哺乳动物的特点，比如它们通常会照顾自己的幼崽，这在非哺乳动物中是不常见的。\n" +
  "\n" +
  "4. **进化关系**：科学研究表明，鲸鱼是从陆地上的四足哺乳动物演化而来的，大约在5000万年前开始适应海洋生活。这一过程展示了鲸鱼与其它哺乳动物之间的密切联系。\n" +
  "\n" +
  "综上所述，从生物学分类到生理结构，再到行为习性和进化历史，鲸鱼都符合哺乳动物的定义。因此，我们可以确定地说，鲸鱼确实是哺乳动物。\n" +
  "    "